In [ ]:
import hydra
import torch
from bliss.encoder import Encoder
from hydra import compose, initialize
from hydra.utils import instantiate

In [ ]:
# GPU to be used
device = torch.device('cuda:0')

In [ ]:
# config file containing parameters of models to run
with initialize(config_path="../../sdss_galaxies/config"):
    cfg = compose("config", overrides=[])

In [ ]:
# example cfg (hydra+OmegaConf, see: https://hydra.cc/)
import omegaconf
omegaconf.OmegaConf.to_container(cfg.models.detection_encoder)

In [ ]:
# load inference model
location = instantiate(cfg.models.detection_encoder).to(device).eval()
location.load_state_dict(
    torch.load(cfg.plots.location_checkpoint, map_location=location.device)
)

binary = instantiate(cfg.models.binary).to(device).eval()
binary.load_state_dict(torch.load(cfg.plots.binary_checkpoint, map_location=device))

galaxy = instantiate(cfg.models.galaxy_encoder).to(device).eval()
galaxy.load_state_dict(torch.load(cfg.plots.galaxy_checkpoint, map_location=device))

encoder = Encoder(
    location.eval(),
    binary.eval(),
    galaxy.eval(),
    n_images_per_batch=10,
    n_rows_per_batch=15,
)
encoder = encoder.to(device)

In [ ]:
# load decode rmodel 
decoder = instantiate(cfg.models.decoder).to(device).eval()

In [ ]:
# load images
images = torch.load('../../sdss_galaxies/.pt')
images.shape # (batch_size, 1, image_size, image_size)

In [ ]:
# obtain prediction (variational mode) per tile
tile_est = encoder.variational_mode(images, background)

In [ ]:
# examine shapes of locs in tile_est, these are normalized [0,1]x[0,1] on tile


In [ ]:
# per tile catalogs can easy be converted to full catalog on images 


In [ ]:
# plot locations on true image from predicted catlaog


In [ ]:
# plot reconstruction


In [ ]:
# NOTE: Samples can be drawn from variational distribution too instead of using
# variational distribution
